In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.bar(df, x="sex", y="total_bill", color="smoker", barmode="group",
             facet_row="time", facet_col="day",
             category_orders={"day": ["Thur", "Fri", "Sat", "Sun"],
                              "time": ["Lunch", "Dinner"]})
fig.show()

In [ ]:
pandas.DataFrame.from_dict(data, orient='columns', dtype=None, columns=None)

In [ ]:
 df = pandas.read_csv("count_vectors_all.csv")